# Co-occurrence network analysis tutorial

In this notebook we will illustrate how interactive exploration and analysis of the [CORD-19](https://www.kaggle.com/allen-institute-for-ai/CORD-19-research-challenge) dataset can be performed using the `cord19kg` package. The exploration and analysis techniques presented here focus on named entities and their co-occurrence in the scientific articles constituting the dataset.

The input data for this notebook contains the named entities extracted from a small selection of 20 articles representing 10 different entity types (i.e. proteins, chemicals, drugs, diseases, condtions, organs, organisms, pathways, cell types, cell compartments). The article selection corresponds to 20 most relevant articles to the query _"Glucose is a risk factor of COVID-19"_ obtained using the article search model [ref to blue brain search](). The entity extraction was performed using the Named Entity Recognition (NER) techniques also included in [ref to blue brain search]().

The `cord19kg` package provides a set of tools for interactive literature exploration through the named entity co-occurrence analysis consisting of the following steps:

1. __Data preparation__ step converts raw mentions into aggregated entity occurrence statistics.
2. __Data curation__ step allows the user to manage extracted entities: modify, filter them and link to the ontology.
3. __Network generation__ step allows creating entity co-occurrence networks based on paper-, section- and paragraph-level co-occurrence relations between entities. These entity relations are quantified using mutual-information-based scores (pointwise mutual information and its normalized version).
4. __Network visualization and analysis__ step allows the user to perform interactive network visualization, edit network elements and perform its analysis (spanning tree, mutual-information based shortest paths between entities, etc).

In [ ]:
import json
import zipfile

import pandas as pd

import dash_cytoscape as cyto

from cord19kg.utils import (generate_curation_table,
                            link_ontology,
                            generate_comention_analysis)
from cord19kg.apps.curation_app import curation_app
from cord19kg.apps.visualization_app import visualization_app

In [ ]:
cyto.load_extra_layouts()

## 1. Data preparation

The input dataset contains occurrences of different terms in paragraphs of scientific articles from the CORD-19 dataset previously extracted by means of a NER model.

In [ ]:
data = pd.read_csv("../data/Glucose_risk_20_papers.csv")

In [ ]:
data.sample(5)

In [ ]:
# %%time
# print("Decompressing the input data file...")
# with zipfile.ZipFile("../data/Glucose_risk_3000_papers.csv.zip", 'r') as zip_ref:
#     zip_ref.extractall("../data/")
# data = pd.read_csv("../data/Glucose_risk_3000_papers.csv")
# print("Done.")

On the first preparation step, we group and aggregate the input data by unique entities.

In [ ]:
%%time
print("Prepating curatation data...")
curation_input_table, factor_counts = generate_curation_table(data)
print("Done.")

The resulting dataframe contains a row per unique named entity together with the following occurrence data: 
- sets of unique paragraphs, papers, sections, where the corresponding entity is mentioned (`paper`, `section`, `paragraph` columns);
- number of total entity occurrences (the `raw_frequency` column);
- number of unique papers where it occurs (the `paper_frequency` column);
- unique entity types assigned by the NER model (the `entity_type` column, multiple types are possible).
- raw entity types assigned by the NER model with the multiplicity of thier occurrence (the `raw_entity_types` column).


In [ ]:
curation_input_table.sample(5)

The second output of the data preparation step outputs the counts of different instances of occurrence factors: number of distinct papers/sections/paragraphs in the input corpus.

In [ ]:
factor_counts

## 2. Data curation

### Loading the NCIT ontology linking data

To group synonymical entities in the previously extracted table (e.g. `ace2`, `ace-2`, `angiotensin-converting enzyme 2`), as well as assign additional semantics to these entities (e.g. human-readable definition, taxonomy, etc), we peform further _linking_ of the entities to the terms from the [NCIT ontology](https://ncithesaurus.nci.nih.gov/ncitbrowser/).

To be able to perform such ontology linking, we load some additional (pre-computed using ML-based linking models) data.

In [ ]:
%%time
print("Loading the ontology linking data...")
    
print("\tDecompressing the input data file...")
with zipfile.ZipFile("../data/NCIT_ontology_linking_3000_papers.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("../data/")

print("\tLoading the linking dataframe in memory...")
ontology_linking = pd.read_csv("../data/NCIT_ontology_linking_3000_papers.csv")

print("\tLoading ontology type mapping...")
with open("../data/NCIT_type_mapping.json", "rb") as f:
    type_mapping = json.load(f)
print("Done.")

The ontology linking table contains the following columns:
- `mention` entity mentioned in the text
- `concept` ontology concept linked to the entity mention
- `uid` unique identifier of the ontology concept
- `definition` definition of the concept
- `taxonomy` taxonomy of semantic types associated with the concept

In [ ]:
ontology_linking.sample(5)

### Interactive curation of  entity occurrence data

The package provides an interactive entity curation app that allows the user to visualize the entity occurrence data, modify it, perform ontology linking (see `Link to NCIT ontology` button), filter short or unfrequent entities.

The field `Keep` allows specifying a set of entities that must be kept in the dataset at all times (even if they don't satisfy the selected filtering criteria).

Finally the value specified in the `Generate Graphs from top 500 frequent entities` field corresponds to the number of top entities (by the frequency of their occurrence in papers) to be included in the co-occurrence network.

We load the prepared data table into the curation app as follows:

In [ ]:
curation_app.set_table(curation_input_table.copy())

We can specify the default entities to keep.

In [ ]:
default_entities_to_keep = ["glucose", "covid-19"]
curation_app.set_default_terms_to_include(default_entities_to_keep)

Finally, we set the ontology linking callback to be fired upon a click on the `Link to NCIT ontology` button.

In [ ]:
curation_app.set_ontology_linking_callback(lambda x: link_ontology(ontology_linking, type_mapping, x))

### Launch the curation app

The application can be launched either inline (inside the current notebook) as below.

In [ ]:
curation_app.run(port=8070, mode="inline")

Or it can be opened externally (by the URL that you can open in a separate tab of your browser, try uncommenting, executing and doing Ctrl+Click on the displayed URL).

In [ ]:
# curation_app.run(port=8070, mode="external")

## 3. Co-occurrence network generation

Current curation table displayed in the curation app can be extracted using the `get_curated_table` method.

In [20]:
curated_occurrence_data = curation_app.get_curated_table()
curated_occurrence_data.head(5)

,aggregated_entities,paragraph,paper,section,raw_entity_types,uid,definition,taxonomy,paper_frequency,entity_type,entity_type_label
entity,,,,,,,,,,,
4e-bp1,[4e-bp1],[214924:The Interplay Between Covid-19 And Amp...,[214924],[214924:The Interplay Between Covid-19 And Amp...,"[PROTEIN, PROTEIN]",NaN,NaN,NaN,1,PROTEIN,PROTEIN
ace inhibitor,[acei],[184360:Combined Therapeutic Potential Targeti...,"[184360, 197804]","[184360:Conclusion, 184360:Aceis And Arbs , 18...","[PROTEIN, DRUG, DRUG, DRUG, DRUG, DRUG, PROTEI...",http://purl.obolibrary.org/obo/NCIT_C247,Any substance that inhibits angiotensin-conver...,"[('http://purl.obolibrary.org/obo/NCIT_C270', ...",2,DRUG,DRUG
acetaminophen,"[acetaminophen, paracetamol]",[197804:Management Of Children And Young Peopl...,"[197804, 179426]","[179426:Blood Glucose Monitoring , 197804:Mana...","[CHEMICAL, DRUG, CHEMICAL, DRUG, DRUG, DRUG]",http://purl.obolibrary.org/obo/NCIT_C198,A p-aminophenol derivative with analgesic and ...,"[('http://purl.obolibrary.org/obo/NCIT_C2356',...",2,DRUG,DRUG
acute lung injury,"[ali, lung injury]","[184360:Caption:71, 184360:Pathophysiology Of ...","[214924, 184360, 197804, 179426]","[214924:The Immune Response To Sars-Cov-2 , 19...","[DISEASE, DISEASE, DISEASE, DISEASE, DISEASE, ...",http://purl.obolibrary.org/obo/NCIT_C155766,A finding of acute onset of bilateral pulmonar...,[('http://purl.obolibrary.org/obo/NCIT_C45233'...,4,DISEASE,DISEASE
acute respiratory distress syndrome,"[acute respiratory distress syndrome, ards]","[184360:Caption:71, 184360:Pathophysiology Of ...","[129074, 211125, 214924, 184360, 179426, 197804]","[214924:Introduction, 214924:The Immune Respon...","[DISEASE, DISEASE, DISEASE, DISEASE, DISEASE, ...",http://purl.obolibrary.org/obo/NCIT_C3353,Progressive and life-threatening pulmonary dis...,[('http://purl.obolibrary.org/obo/NCIT_C28193'...,6,DISEASE,DISEASE


Before we can proceed we need to convert paper/section and paragraph columns into `set`.

In [ ]:
curated_occurrence_data["paper"] = curated_occurrence_data["paper"].apply(set)
curated_occurrence_data["paragraph"] = curated_occurrence_data["paragraph"].apply(set)
curated_occurrence_data["section"] = curated_occurrence_data["section"].apply(set)

We can also retreive current values of the `Keep` field (these entities will be also included in the resulting co-occurrence network).

In [ ]:
curation_app.get_terms_to_include()

And the number of top frequent entities to use for network generation.

In [ ]:
curation_app.n_most_frequent

### Generating co-occurrence networks

In the cell below we generate two co-occurrence networks: for paper- and paragraph-based entity co-occurrences. Along with the network generation the `generate_comention_analysis` function 
- computes node centrality metrics (such as degree, RageRank)
- computes co-occurrence statistics (such as frequency, pointwise mutual information and normalized pointwise mutual information) and assignes them as weights to the corresponding edges
- performs entity community detection based on different co-occurrence statistics
- computes mutual-information-based minimum spanning trees.

In [ ]:
%%time
type_data = curated_occurrence_data[["entity_type"]].rename(columns={"entity_type": "type"})

graphs, trees = generate_comention_analysis(
    curated_occurrence_data,  factor_counts,
    n_most_frequent=curation_app.n_most_frequent if curation_app.n_most_frequent else 100,
    type_data=type_data, 
    factors=["paper", "paragraph"],
    keep=curation_app.get_terms_to_include(),
    cores=10)
print("Done.")

## 4. Network visualization and analysis

### Loading the generated graphs into the visualization app

In [ ]:
visualization_app.set_graph(
    "Paper-based graph", graphs["paper"],
    tree_object=trees["paper"], default_top_n=100)

visualization_app.set_graph(
    "Paragraph-based graph", graphs["paragraph"],
    tree_object=trees["paragraph"], default_top_n=100)

visualization_app.set_current_graph("Paragraph-based graph")

### Loading papers' meta-data into the app

We now load an additional dataset containing some meta-data on the papers where the entities analyzed in this notebook occur.

In [ ]:
paper_data = pd.read_csv("../data/Glucose_risk_3000_paper_meta_data.csv")
paper_data = paper_data.set_index("id")
paper_data.head(3)

We pass a callback for the lookup of paper meta-data to the visualization app using the `set_list_papers_callback` method.

In [ ]:
def list_papers(paper_data, selected_papers, limit=200):
    selected_paper_data = paper_data.loc[[int(p) for p in selected_papers]].head(200)
    return selected_paper_data.to_dict("records")

visualization_app.set_list_papers_callback(lambda x: list_papers(paper_data, x))

The ontology linking process described above is noisy, therefore, we would like to keep a possibility of accessing, the raw entities that were linked to particular ontology concepts. For this we define the function `get_aggregated_entities` that retreives such raw entities and we pass it to the visualization app using the `set_aggregated_entities_callback` method.

In [ ]:
def top_n(data_dict, n, smallest=False):
    """Return top `n` keys of the input dictionary by their value."""
    df = pd.DataFrame(dict(data_dict).items(), columns=["id", "value"])
    if smallest:
        df = df.nsmallest(n, columns=["value"])
    else:
        df = df.nlargest(n, columns=["value"])
    return(list(df["id"]))


def get_aggregated_entities(entity, n):
    if "aggregated_entities" in curated_occurrence_data.columns:
        aggregated = curated_occurrence_data.loc[entity]["aggregated_entities"]
    else:
        aggregated = [entity]
    if curation_input_table is not None:
        df = curation_input_table.set_index("entity")
        if entity in curated_occurrence_data.index:
            freqs = df.loc[aggregated]["paper_frequency"].to_dict()
        else:
            return {}
    else:
        df = data.copy()
        df["entity"] = data["entity"].apply(lambda x: x.lower())
        freqs = df[df["entity"].apply(lambda x: x.lower() in aggregated)].groupby("entity").aggregate(
            lambda x: len(x))["entity_type"].to_dict()
    if len(freqs) == 0:
        return {}
    return {e: freqs[e] for e in top_n(freqs, n)}

visualization_app.set_aggregated_entities_callback(
    lambda x: get_aggregated_entities(x, 10))

Finally, we create a dictionary `definitions` that will serve the visualization app as the lookup table for accessing the definitions of different ontology concepts.

In [ ]:
definitions = ontology_linking[["concept", "definition"]].groupby(
    "concept").aggregate(lambda x: list(x)[0]).to_dict()["definition"]
visualization_app.set_entity_definitons(definitions)

### Launching the visualization app

As before, the interactive graph visualization app can be launched in two modes: inline and external. Here we recommend the external mode for better user experience.

In [ ]:
visualization_app.run(port=8079, mode="external")

### Guide to the visualization app interface

The visualization app contains three panels highlighted below:

<img src="../figures/visualization_app_structure.png" alt="Drawing" style="width: 800px;"/>


The graph displayed by the app corresponds to the mutual-information-based minimum spanning tree computed on the selected set of nodes.

#### 1. Search panel

The element on the figure below allows us to search for nodes (note that at the moment the result of the search is simply highlighted in the graph and not zoomed in).
<img src="../figures/1_search.png" alt="Drawing" style="width: 600px;"/>

The graph displayed in the app at any time can be downloaded using different formats (including the graph format `gml`).
<img src="../figures/1_download.png" alt="Drawing" style="width: 600px;"/>


#### 2.  View control panel

This panel expands when the `Controls` button is clicked. It contains three tabs allowing to control different aspects of the graph view.

The first tab allows the user to switch between different loaded graphs, filter nodes/edges by different criteria (frequencies, weights, nodes types, communities)

<img src="../figures/2_view.png" alt="Drawing" style="width: 400px;"/>

The second tab allows to change the layout of the graph and highlighting color scheme

<img src="../figures/2_layouts.png" alt="Drawing" style="width: 400px;"/>

The third tab allows to search for different paths in the graph (the paths are found by minimizing the distance score based on the mutual information)

<img src="../figures/2_paths.png" alt="Drawing" style="width: 400px;"/>

#### 3. Details panel

This panel allow the user to view and interact with different graph elements. It consists of 4 tabs.

The legend tab provides the interpretation of the node colors

<img src="../figures/3_legend.png" alt="Drawing" style="width: 800px;"/>

The details tab allows to access the knowledge associated to nodes and edges of the selected graph elements. For graph nodes, for example, we can see the definition of the associated ontology concept, all the papers where the concept is mentioned, all the raw entities associated to the concept.

<img src="../figures/3_detail.png" alt="Drawing" style="width: 800px;"/>

The edit tab allows to edit different graph elements (remove nodes and edges, rename nodes and merge multiple nodes). Editing is available in two modes: the editing mode actually modifies the uderlying graph object, while the masking mode simply acts on the current graph view (when the view is refreshed the edits are reset).

<img src="../figures/3_edit.png" alt="Drawing" style="width: 800px;"/>

Finally, the neighbors tab allows the user to access to the top N closest neighbors of the selected node by the largest mutual information score (normalized pointwise mutual information).

<img src="../figures/3_neighbors.png" alt="Drawing" style="width: 800px;"/>